<a href="https://colab.research.google.com/github/johannnamr/Discrepancy-based-inference-using-QMC/blob/main/Inference/Bv-beta-distribution/bibeta_optim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bivariate Beta distribution

### Mount Drive

In [ ]:
# mount my drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The following path has to be adjusted if necessary:

In [ ]:
%run "/content/drive/My Drive/Colab Notebooks/Utils.ipynb"
%run "/content/drive/My Drive/Colab Notebooks/Plot_fcts.ipynb"
%run "/content/drive/My Drive/Colab Notebooks/ot_slicedW.ipynb"

Set path for saving the results (adjust if necessary):

In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/Paper/Inference/'

Imports:

In [ ]:
import numpy as np
from scipy import optimize

## Sampling

In [ ]:
m = 2**16             # number of true samples
m_sub = 2**10         # number of subsamples
n = np.round((2**9)**(3/4),0).astype(int)     # number of simulated samples
#n = 2**9              # number of simulated samples
theta = (1,1,1,1,1)   # true theta
d = 2                 # data dimensions
l =1.5*d**(1/2)       # lengthscale of the Gaussian kernel for MMD^2
e = 5*d               # regularisation parameter for Sinkhorn divergence
maxiter = 3000        # maximum number of optimisation iterations
div_type = 'mmd2'     # 'mmd2', 'wasserstein', 'sliced_wasserstein' or 'sinkhorn'
method = 'RQMC'         # 'MC' or 'RQMC'
rep = 7               # repetition no.

In [ ]:
seed = [133,134,135,123,124,125,113,114,115,103]

In [ ]:
np.random.seed(seed[rep])

In [ ]:
i_theta,_ = divmod(theta,np.ones(5)) 
i_theta.astype(int)
qmc = qmcpy.Halton(np.sum(i_theta.astype(int)),seed=7)

In [ ]:
y = sample_bibeta('MC',m,theta)

MMD$^2_V$ function dependent on $\theta$:

In [ ]:
def opt_sample_qmc_gamma(alpha,n):

  'Function to simulate from a gamma distribution with \alpha<1 and \beta=1 using QMC points'

  # get b
  b = (alpha+np.e)/np.e

  # fix QMC sequence
  qmc = qmcpy.Halton(3)

  # generate samples
  i = 0
  num = 0
  gamma = np.array([])
  while num < n:
    # get qmc samples
    omega = np.squeeze(qmc.gen_samples(n_min=i,n_max=i+1))
    # accept-reject algorihtm
    y = b*omega[0]
    if y<=1:
      x = y**(1/alpha)
      if omega[1]<=np.e**(-x):
        gamma = np.append(gamma,x)
        num += 1 # counts accepted points
    else:
      x = -np.log((b-y)/alpha)
      if omega[2]<=x**(alpha-1):
        gamma = np.append(gamma,x)
        num += 1 # counts accepted points
    i += 1 # counts number of simulated points

  return gamma

In [ ]:
def opt_sample_bibeta(theta, *params):

  _,method,_,n,_,_,_ = params

  # split theta into integer and decimal parts
  i_theta,d_theta = divmod(theta,np.ones(5)) 
  i_theta = i_theta.astype(int)
  p = np.sum(i_theta)

  # sample uniforms
  if method == 'MC':
    unif = np.random.rand(n,p)
  if method == 'QMC':
    unif = qmc.gen_samples(n)
  if method == 'RQMC':
    unif = qmcpy.Halton(p).gen_samples(n)

  # initialise
  utild = np.zeros([n,5])
  x = np.zeros((n,2))

  # logs on uniforms
  logunif = np.log(unif)

  # get \tilde{u}
  j=0
  for i in range(5):
    sum = np.zeros(n)
    if i_theta[i]!=0:
      for k in range(i_theta[i]):
        sum[:] += logunif[:,k+j]
      utild[:,i] = -sum
    if d_theta[i]!=0:
      if method=='QMC' or method=='RQMC':
        utild[:,i] += opt_sample_qmc_gamma(d_theta[i],n)
      else:
        utild[:,i] += np.random.gamma(d_theta[i],1,n)
    j += i_theta[i]

  # generator
  x1 = np.sum(np.vstack([utild[:,0],utild[:,2]]),axis=0)/np.sum(np.vstack([utild[:,0],utild[:,2],utild[:,3],utild[:,4]]),axis=0)
  x2 = np.sum(np.vstack([utild[:,1],utild[:,3]]),axis=0)/np.sum(np.vstack([utild[:,1],utild[:,2],utild[:,3],utild[:,4]]),axis=0)

  return np.vstack([x1,x2]).T

In [ ]:
def opt_mmd2(theta,*params):

  _,_,m,n,y,l,_ = params

  # subsample from the true data
  sub = np.random.choice(np.arange(y.shape[0]),m)
  y_sub = y[sub,:]

  # generate samples
  x = opt_sample_bibeta(theta,*params)

  # gaussian kernel
  r = distance.cdist(x,x,'sqeuclidean')
  kxx = np.exp(-(1/(2*l**2))*r)
  r = distance.cdist(x,y_sub,'sqeuclidean')
  kxy = np.exp(-(1/(2*l**2))*r)
  r = distance.cdist(y_sub,y_sub,'sqeuclidean')
  kyy = np.exp(-(1/(2*l**2))*r)

  # first sum
  sum1 = np.sum(kxx)
  # second sum
  sum2 = np.sum(kxy)
  # third sum
  sum3 = np.sum(kyy)
  
  mmd2 = (1/n**2)*sum1-(2/(m*n))*sum2+(1/m**2)*sum3

  return mmd2

In [ ]:
def opt_w(theta,*params):

  _,_,m,n,y,_,_ = params

  # subsample from the true data
  sub = np.random.choice(np.arange(y.shape[0]),m)
  y_sub = y[sub,:]

  # generate samples
  x = opt_sample_bibeta(theta,*params)

  # equal weights
  a = np.ones((n,)) / n 
  b = np.ones((m,)) / m
    
  #MC and RQMC
  M = ot.dist(x, y_sub, 'euclidean')
  M /= M.max()
  w = ot.emd2(a, b, M)

  return w

In [ ]:
def opt_sw(theta,*params):

  _,_,m,n,y,_,_ = params

  # subsample from the true data
  sub = np.random.choice(np.arange(y.shape[0]),m)
  y_sub = y[sub,:]

  # generate samples
  x = opt_sample_bibeta(theta,*params)

  # equal weights
  a = np.ones((n,)) / n 
  b = np.ones((m,)) / m
    
  #MC and RQMC
  sw = sliced_wasserstein_distance(x,y_sub, 'euclidean', a, b, n_projections=100)

  return sw

In [ ]:
def opt_sink(theta,*params):

  _,_,m,n,y,_,e = params

  # subsample from the true data
  sub = np.random.choice(np.arange(y.shape[0]),m)
  y_sub = y[sub,:]

  # generate samples
  x = opt_sample_bibeta(theta,*params)

  # sinkhorn divergence (equal weights by default)
  sink = ot.bregman.empirical_sinkhorn_divergence(x, y_sub, e, cost='seuclidean', method='sinkhorn')

  return sink

In [ ]:
def opt_bibeta(theta,*params):

  div_type,_,_,_,_,_,_ = params

  # calculate divergence
  if div_type=='mmd2':
    div = opt_mmd2(theta,*params)
  if div_type=='wasserstein':
    div = opt_w(theta,*params)
  if div_type=='sliced_wasserstein':
    div = opt_sw(theta,*params)
  if div_type=='sinkhorn':
    div = opt_sink(theta,*params)  
  
  return div

Start optimisation using differential evolution:

In [ ]:
params = (div_type,method,m_sub,n,y,l,e)
minimizer_kwargs =  { 'args': params }
bounds = ((0,2),(0,2),(0,2),(0,2),(0,2))

In [ ]:
t = TicToc()
t.tic()
opt = optimize.differential_evolution(opt_bibeta, bounds, args=params, maxiter=maxiter)
time = t.tocvalue()

In [ ]:
time

10677.739475998

In [ ]:
time/60**2

2.9660387433327777

In [ ]:
opt

     fun: 3.125200466191913e-07
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 225201
     nit: 3000
 success: False
       x: array([0.61 , 1.936, 1.221, 0.592, 1.242])

Calculate the precise loss for the estimated parameters:

In [ ]:
params_precise = params[:3] + (2**14,) + params[4:]

In [ ]:
loss = opt_bibeta(opt.x,*params_precise)

Save the results:

In [ ]:
np.savez(path+"bibeta_optim_%s_%s_n=%s_%s.npz" %(params[0],params[1],n,rep),time=time,loss=loss,theta=opt.x,optloss=opt.fun)